In [ ]:
!git clone https://github.com/vladislavkorenik/image-restoration
!mv image-restoration/* ./

In [ ]:
from model import SRResnet
from train_utils import Train

architecture = SRResnet(3, 64)

params = {
    'noise_model': ('multiplicative_bernoulli', 0.8),
    'crop_size': 64,
    'clean_targs': False,
    'lr': 0.001,
    'epochs': 20,
    'bs': 2,
    'lossfn': 'l2',
    'cuda': True
}

In [109]:
trainer = Train(architecture, 'dataset/train', 'dataset/valid', params)
trainer.train()
model = trainer.architecture

In [112]:
import torch
import numpy as np
from data import NoisyDataset
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader

def show(img, a):
    plt.figure()
    if a == 'v':
        npimg = torch.squeeze(img).data.cpu().numpy()
    else:
        npimg = torch.squeeze(img).numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))

In [ ]:
data_ = NoisyDataset('dataset/valid/', crop_size=128, clean_targ=True, train_noise_model=('multiplicative_bernoulli', 0.6))
dl = DataLoader(data_, batch_size=1, shuffle=True)

i = 0
for _list in dl:
    plt.figure()
    pred = model(Variable(_list[0].cuda()))
    show(pred, 'v')
    show(_list[0], 't')
    show(_list[-1], 't')
    if i == 0:
        break